In [ ]:
import pandas as pd
import networkx as nx
import time
passengers_df = pd.read_csv('/content/passengers.csv')
flights_df = pd.read_csv('/content/flights.csv')
canceled_flights_df = pd.read_csv('/content/canceled.csv')
canceled_flights = set(canceled_flights_df)
available_flights_df = flights_df[~flights_df['FID'].isin(canceled_flights)]
G = nx.DiGraph()
for _, row in available_flights_df.iterrows():
    G.add_edge(row['DEP'], row['ARR'],
               flight_id=row['FID'],
               departure_time=row['DEP_TIME'],
               arrival_time=row['ARR_TIME'],
               capacity=row['CAPACITY'])
def find_reallocation(passenger_id, original_flight_id):
    original_flight = flights_df[flights_df['FID'] == original_flight_id].iloc[0]
    source = original_flight['DEP']
    destination = original_flight['ARR']
    original_arrival_time = original_flight['ARR_TIME']
    try:
        paths = nx.single_source_dijkstra_path_length(G, source)
    except nx.NetworkXNoPath:
        return (0, [])
    sorted_paths = sorted(paths.items(), key=lambda x: (len(x[1])-1, abs(x[1][-1]['arrival_time'] - original_arrival_time)))

    for path in sorted_paths:
        if path[-1]['arrival_airport'] == destination:
            flights = [edge[2]['flight_id'] for edge in path]
            if len(flights) <= 3:
                return (len(flights), flights)

    return (0, [])
affected_passengers = passengers_df[passengers_df['FID'].isin(canceled_flights)]
reallocated_passengers = 0
total_layovers = 0
total_time_diff = 0
start_time = time.time()

allot_list = []

for _, row in affected_passengers.iterrows():
    passenger_id = row['PID']
    original_flight_id = row['FID']
    num_flights, reallocation = find_reallocation(passenger_id, original_flight_id)

    allot_list.append([passenger_id, num_flights] + reallocation)

    if num_flights > 0:
        reallocated_passengers += 1
        total_layovers += (num_flights - 1)
        original_arrival_time = flights_df[flights_df['FID'] == original_flight_id].iloc[0]['ARR_TIME']
        final_arrival_time = flights_df[flights_df['FID'] == reallocation[-1]].iloc[0]['ARR_TIME']
        total_time_diff += abs(final_arrival_time - original_arrival_time)

end_time = time.time()
execution_time_ms = (end_time - start_time) * 1000
total_affected_passengers = len(affected_passengers)
average_layovers = total_layovers / reallocated_passengers if reallocated_passengers else 0
average_time_diff = total_time_diff / reallocated_passengers if reallocated_passengers else 0
stats_df = pd.DataFrame({
    'total_affected_passengers': [total_affected_passengers],
    'reallocated_passengers': [reallocated_passengers],
    'average_layovers': [average_layovers],
    'average_time_diff': [average_time_diff],
    'execution_time_ms': [execution_time_ms]
})
stats_df.to_csv('stats.csv', index=False)
allot_df = pd.DataFrame(allot_list, columns=['passenger_id', 'num_flights', 'flight_1', 'flight_2', 'flight_3'])
allot_df.fillna(0, inplace=True)
allot_df.to_csv('allot.csv', index=False)

